Задание. Дерево решений

В нижеприведенном коде

1)	Реализовать функцию расчет критерия Джини
2)	Реализовать расчет критерия прироста информации
3)	Реализовать критерии останова (не менее двух)
4)	Реализовать функцию подсчета метрики модели
5)	Проверить работу самописного дерева для решения задачи классификаци (датасет можно сгенерировать самостоятельно при помощи функции make_classification, либо взять любой набор данных из задания к ЛР4), сравнив полученные результаты с результатами дерева решений из sklearn. Результаты должны совпасть полностью.
